In [2]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
import mlflow
import torch

In [3]:
#Funções para Teste
def testar(nome_modelo,nome_dataset_teste,colunas_estado,colunas_ativos,desvio_padrao = 1e-6):
    #Carregar Modelo
    def carregar_modelo(nome_do_modelo, versao=None):
        #Iniciando MlFlow
        mlflow.set_tracking_uri("http://127.0.0.1:5000")
        if versao:
            model_uri = f"models:/{nome_do_modelo}/{versao}"
        else:
            model_uri = f"models:/{nome_do_modelo}/latest"
        
        # Carregar o modelo
        modelo = mlflow.pytorch.load_model(model_uri)
        return modelo
    modelo = carregar_modelo(nome_modelo)
    
    #Realizar previsões
    def prever(modelo, nome_dataset_teste,colunas_estado,desvio_padrao = 1e-6):
        #Carregar Dataset
        data = pd.read_csv(f'../data/{nome_dataset_teste}')
        X_test =np.array(data.loc[:, colunas_estado])
        #Prever
        modelo.eval()
        previsoes= []
        with torch.no_grad():
            for i in X_test:
                i_tensor = torch.tensor(i,dtype=torch.float32)
                previsao = modelo(i_tensor)
                dist = torch.distributions.Normal(previsao, desvio_padrao)  # Cria a distribuição normal
                action = dist.rsample() 
                previsoes.append(action.detach().numpy())
        return previsoes
    previsoes = prever(modelo,nome_dataset_teste,colunas_estado,desvio_padrao)

    #Criar Gráficos com Plotly
    def criar_grafico(previsoes,nome_dataset_teste,colunas_ativos):
        # Criar uma figura
        df = pd.DataFrame(previsoes,columns=colunas_ativos)
        fig = go.Figure()
        for coluna in colunas_ativos:
            fig.add_trace(go.Scatter(x=df.index, y=df[coluna], mode='lines',name=coluna))    
        fig.show()
    criar_grafico(previsoes,nome_dataset_teste,colunas_ativos)


In [11]:
testar('Magnum_1_1_C','merged_v1_test.csv',['Valor IPCA'],['Ouro','Bitcoin','Ibovespa'])

C:\Users\joaop\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.10.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
